In [1]:
#!pip install -q transformers datasets accelerate 

Load training data; i know the dev set lines up with the validation split from hf so we'll train on a subset of training of equal size (otherwise its an 8hr wait)

## Main idea:
- first assess the 'zero-shot' accuracy of pretrained bert-based on the squadv2 data (inherently can't get null responses so perf should be low) 
- fine tune model on a subset of the data and see the accuracy 
- get a pretrained one on squadv2 and rotely predict 

In [1]:
from datasets import load_dataset
test = load_dataset("squad_v2", split = 'validation') #[0:{5*test.shape[0]}]
train = load_dataset("squad_v2", split = f'train') #truncated to same length as validation for fine-tuning feasibility 
print(train.shape, test.shape)

Found cached dataset squad_v2 (/home/n.nethercott/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)
Found cached dataset squad_v2 (/home/n.nethercott/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


(130319, 5) (11873, 5)


Move the following cell to an external utils.py for final submission 

In [2]:
# preprocessing util here for identifying answer spans 

# Note: had to add a conditional to check for potentially empty strings, i think the tokenizer 
# in the hf tutorial handled this already 

def preprocess_function(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]

        if len(answer["answer_start"])>0:
          start_char = answer["answer_start"][0]
          end_char = answer["answer_start"][0] + len(answer["text"][0])
          sequence_ids = inputs.sequence_ids(i)

          # Find the start and end of the context
          idx = 0
          while sequence_ids[idx] != 1:
              idx += 1
          context_start = idx
          while sequence_ids[idx] == 1:
              idx += 1
          context_end = idx - 1

          # If the answer is not fully inside the context, label it (0, 0)
          if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
              start_positions.append(0)
              end_positions.append(0)
          else:
              # Otherwise it's the start and end token positions
              idx = context_start
              while idx <= context_end and offset[idx][0] <= start_char:
                  idx += 1
              start_positions.append(idx - 1)

              idx = context_end
              while idx >= context_start and offset[idx][1] >= end_char:
                  idx -= 1
              end_positions.append(idx + 1)
        else:
          start_positions.append(0)
          end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

2 training approaches:
- actually start from ground zero with bert-base-(un)cased and fit a model 
- use one pretrained on squadv1 and fine tune to recognize unanswerable questions

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-cased")

2023-05-25 16:31:26.945828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 16:31:27.091785: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 16:31:31.943118: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/10.2.0/lib:/cm/local/apps/gcc/10.2.0/lib64:/cm/local/apps/gcc/10.2.0

In [4]:
text = "let me check to see the tokenization is predictable"  #; nice 
tokenizer.decode(tokenizer(text, text).input_ids)

'[CLS] let me check to see the tokenization is predictable [SEP] let me check to see the tokenization is predictable [SEP]'

In [5]:
#preprocess train and test with this bad boy 
from functools import partial 

train = train.map(partial(preprocess_function, tokenizer=tokenizer), batched=True)
test = test.map(partial(preprocess_function, tokenizer=tokenizer), batched=True)

train

Loading cached processed dataset at /home/n.nethercott/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-aa5b1480767990ea.arrow
Loading cached processed dataset at /home/n.nethercott/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d/cache-2f4362eb6d8d88b7.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 130319
})

Data prep:
we can use the hf high-level trainer and such but honestly that confuses me & kinda obscures the training process a bit. Instead, I'll setup the dataloader (batches samples for GD algo and also nice since you can't allocate sufficient memory) for all the tensors all at once  

In [6]:
from torch.utils.data import DataLoader

class TorchDataset:
  def __init__(self, data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, i):
      return self.data[i]

# turn list of dicts into dict of lists 
def custom_collate(data): 
    return {k:[d[k] for d in data] for k in data[0].keys()}

train_data = TorchDataset(train)
train_loader = DataLoader(train_data, batch_size = 8, shuffle = True, collate_fn=custom_collate)

test_data = TorchDataset(test)
test_loader = DataLoader(test_data, batch_size = 8, collate_fn=custom_collate)

make sure that worked as expected

In [7]:
len(next(iter(train_loader))['id'])

8

predict over dev to generate baseline performance

In [9]:
from tqdm import tqdm 
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

#model.eval()

preds = {}
for batch in tqdm(test_loader):
    input_ids = torch.tensor(batch['input_ids'], device = device)
    attention_mask = torch.tensor(batch['attention_mask'], device=device)
    start_positions = torch.tensor(batch['start_positions'], device = device)
    end_positions = torch.tensor(batch['end_positions'], device=device)

    out = model(input_ids=input_ids, 
                attention_mask = attention_mask,
                start_positions = start_positions,
                end_positions = end_positions,
               )

    #recover prediction span 
    start = out.start_logits.argmax(dim=1).cpu().detach()
    end = out.end_logits.argmax(dim=1).cpu().detach()

    for i, p in enumerate(list(zip(start, end))):
        decoded = tokenizer.decode(input_ids[i,p[0]:p[1]+1])
        preds[batch['id'][i]] = decoded if decoded != '[CLS]' else ''

    #break 

100%|██████████| 1485/1485 [01:31<00:00, 16.19it/s]


link drive to save the baseline predictions

In [11]:
import json 
with open("preds_baseline.json", "w") as f:
    json.dump(preds, f)

In [12]:
# !wget https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/ -O evaluation.py
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O dev-v2.0.json

In [13]:
!python3 evaluation.py ./dev-v2.0.json preds_baseline.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 25.191611218731577,
  "f1": 27.015494626847186,
  "total": 11873,
  "HasAns_exact": 0.0,
  "HasAns_f1": 3.652997251106355,
  "HasAns_total": 5928,
  "NoAns_exact": 50.31118587047939,
  "NoAns_f1": 50.31118587047939,
  "NoAns_total": 5945
}


## okay now let's fine tune the model on the training data and see if we improve 

In [16]:
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm 
import torch 

device = "cuda" if torch.cuda.is_available() else "cpu"

optimizer = AdamW(model.parameters(), lr=3e-5)

num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

model.to(device)
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [15]:
len([1 for i in train['answers'] if len(i['answer_start'])==0])/len(train)

0.3337809528925176

would be nice to handle tensors and stuff in collator so we could use the **kwargs syntax but hey

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    for batch in train_loader:
        try:
            input_ids = torch.tensor(batch['input_ids'], device = device)
            attention_mask = torch.tensor(batch['attention_mask'], device=device)
            start_positions = torch.tensor(batch['start_positions'], device = device)
            end_positions = torch.tensor(batch['end_positions'], device = device)

            outputs = model(input_ids=input_ids, 
                            attention_mask = attention_mask,
                            start_positions = start_positions,
                            end_positions = end_positions,
                            )

            loss = outputs.loss
            loss.backward()

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

        except:
            pass

  0%|          | 0/32580 [00:00<?, ?it/s]

In [ ]:
#!mkdir model-weights
torch.save(model.state_dict(), './model-weights/bert-again-large')

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

preds = {}
for batch in tqdm(test_loader):
  input_ids = torch.tensor(batch['input_ids'], device = device)
  attention_mask = torch.tensor(batch['attention_mask'], device=device)

  out = model(input_ids=input_ids, attention_mask = attention_mask)

  #recover prediction span 
  start = out.start_logits.argmax(dim=1).cpu().detach()
  end = out.end_logits.argmax(dim=1).cpu().detach()

  for i, p in enumerate(list(zip(start, end))):
    decoded = tokenizer.decode(input_ids[i,p[0]:p[1]+1])
    preds[batch['id'][i]] = decoded if decoded != '[CLS]' else ''


  0%|          | 0/1485 [00:00<?, ?it/s]

In [20]:
import json 
with open("./preds_large.json", "w") as f:
    json.dump(preds, f)

In [34]:
null_preds = {k:'' for k in preds.keys()}
with open("preds_dummy.json", "w") as f:
    json.dump(null_preds, f)

In [21]:
!python3 evaluation.py ./dev-v2.0.json preds_large.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "exact": 67.59033100311632,
  "f1": 71.36014963533503,
  "total": 11873,
  "HasAns_exact": 63.46153846153846,
  "HasAns_f1": 71.01198660936785,
  "HasAns_total": 5928,
  "NoAns_exact": 71.70731707317073,
  "NoAns_f1": 71.70731707317073,
  "NoAns_total": 5945
}


In [33]:
!rm preds_dummy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
